In [50]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")

f = open('11-0.txt', 'rb')

sentences = []

for sentence in f:
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = sentence.decode('ascii','ignore')
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()

total_data = ' '.join(sentences)

char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print(vocab_size)


56


In [2]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

print(index_to_char)
train_X = 'appl'
train_y = 'pple'

seq_length = 60

n_samples = int(np.floor((len(total_data) - 1) / seq_length))
print ('샘플의 수 : {}'.format(n_samples))

{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: "'", 7: '(', 8: ')', 9: '*', 10: ',', 11: '-', 12: '.', 13: '/', 14: '0', 15: '1', 16: '2', 17: '3', 18: '4', 19: '5', 20: '6', 21: '7', 22: '8', 23: '9', 24: ':', 25: ';', 26: '?', 27: '[', 28: ']', 29: '_', 30: 'a', 31: 'b', 32: 'c', 33: 'd', 34: 'e', 35: 'f', 36: 'g', 37: 'h', 38: 'i', 39: 'j', 40: 'k', 41: 'l', 42: 'm', 43: 'n', 44: 'o', 45: 'p', 46: 'q', 47: 'r', 48: 's', 49: 't', 50: 'u', 51: 'v', 52: 'w', 53: 'x', 54: 'y', 55: 'z'}
샘플의 수 : 2658


In [46]:
train_X = []
train_y = []
print(char_to_index)
for i in range(n_samples):
  X_sample = total_data[i * seq_length : (i+1) * seq_length]
  X_encoded = [char_to_index[x] for x in X_sample]
  train_X.append(X_encoded)

  # 오른쪽으로 1칸 쉬프트
  y_sample = total_data[i * seq_length + 1: (i + 1) * seq_length + 1]
  y_encoded = [char_to_index[c] for c in y_sample]
  train_y.append(y_encoded)

# print(train_X)
# print(train_y[0])
# print('-'*50)
# print([index_to_char[i] for i in train_X[0]])
# print([index_to_char[i] for i in train_y[0]])

train_X = to_categorical(train_X)
train_y = to_categorical(train_y)
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩






{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}
train_X의 크기(shape) : (2658, 60, 56)
train_y의 크기(shape) : (2658, 60, 56)


In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed
hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=0)


In [81]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)]

    y_char = [index_to_char[ix[-1]]]
    print(y_char)
    
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0],1)
        y_char.append(index_to_char[ix[-1]])

    return ('').join(y_char)

result = sentence_generation(model, 60)
print(result)

['4']
4 down his frown was the only on the door catried i cholded 4 down his frown was the only on the door catried i cholded t
